In [2]:
!pip install MPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 693.6 kB/s eta 0:00:00a 0:00:01


In [5]:
pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 641.2 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for mpi4py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [148 lines of output]
      running bdist_wheel
      running build
      running build_src
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/mpi4py
      copying src/mpi4py/bench.py -> build/lib.linux-x86_64-cpython-311/mpi4py
      copying src/mpi4py/__init__.py -> build/lib.linux-x86_64-cpython-311/mpi4py
      copying src/mpi4py/__main__.py -> build/lib.linux-x86_64-cpython-311/mpi4py
      copying src/mpi4py/run.py -> build/lib.linux-x86_64-cpython-311/mpi4py
      creating build/lib.linux-x86_64-cpython-311/mpi4py/futures
      copy

In [4]:
conda install mpi4py

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mpi4py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mpich-4.1.2                |     h846660c_100        24.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        24.5 MB

The following NEW packages will be INSTALLED:

  mpi                conda-forge/linux-64::mpi-1.0-mpich 
  mpi4py             conda-forge/linux-64::mpi4py-3.1.5-py311he01e52e_0 
  mpich              conda-forge/linux-64::mpich-4.1.2

In [9]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
N = 1000  # You should define the value of N

sum_xy = []
executiontime = 0.0

if rank != 0:
    start = MPI.Wtime()
    a = comm.recv(source=0)
    b = comm.recv(source=0)
    comm.send(a + b, dest=0, tag=1)
    end = MPI.Wtime()
    comm.send(end - start, dest=0, tag=2)
    print("Time taken by worker", rank, "is:", end - start)
else:
    start = MPI.Wtime()
    x = np.random.randint(100, size=N)
    y = np.random.randint(100, size=N)
    split_x = np.array_split(x, size)
    split_y = np.array_split(y, size)

    if rank == 0:
        for worker in range(1, size):
            comm.send(split_x[worker], dest=worker)
            comm.send(split_y[worker], dest=worker)
            sum_xy.extend(list(comm.recv(source=worker, tag=1)))
            executiontime += comm.recv(source=worker, tag=2)

    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker", rank, "is:", end - start)

if rank == 0:
    print("Total execution time:", executiontime)

Time taken by worker 0 is: 0.009351618999971834
Total execution time: 0.009351618999971834


In [4]:
from mpi4py import MPI
import numpy as np
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
N = int(1e4)
average = None
average_x = []
executiontime = 0
if rank != 0:
    start = MPI.Wtime()
    a = comm.recv(source = 0)
    comm.send(np.sum(a)/len(a), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start, dest=0,tag=2)
    print("Time taken by worker", rank, "is: ", end - start)
else:
    start = MPI.Wtime()
    x = np.random.randint(100,size=N)
    split_x = np.array_split(x,size)
    if rank == 0:
        average_x.append(np.sum(split_x[rank])/len(split_x[rank]))
    for worker in range(1,size):
        comm.send(split_x[worker], dest = worker)
        average_x.append(comm.recv(source = worker, tag=1))
        executiontime+=comm.recv(source=worker,tag=2)
    average = np.average(average_x)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker", rank, "is: ", end - start)

Time taken by worker 0 is:  0.00042961799999829964


In [5]:
from mpi4py import MPI
import numpy as np
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
N = int(1e4)
VecMult = []
executiontime = 0
if rank != 0:
    start = MPI.Wtime()
    A = comm.recv(source = 0)
    b = comm.recv(source = 0)
    comm.send(np.matmul(A,b), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker",rank,"is: ", end - start)
else:
    start = MPI.Wtime()
    A = np.random.randint(100,size=(N,N))
    b = np.random.randint(100,size=(N,1))
    split_A = np.array_split(A,size)
    if rank == 0:
        out = np.matmul(split_A[rank],b)
        VecMult.extend(out.flatten().tolist())
    for worker in range(1,size):
        comm.send(split_A[worker], dest = worker)
        comm.send(b, dest = worker)
        out = comm.recv(source = worker, tag=1)
        VecMult.extend(out.flatten().tolist())
        executiontime+=comm.recv(source = worker, tag=2)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker", rank, "is: ", end - start)
    

Time taken by worker 0 is:  1.2265770329998986


In [9]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
A = None
B = None
N = int(1e4)
C = np.zeros((N,N))
if rank !=0:
    B = np.empty((N,N))
else:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
Arecv = np.empty((int(N/size),N))
start = MPI.Wtime()
comm.Scatter(A,Arecv, root=0)
comm.Bcast(B,root=0)
comm.Gather(np.matmul(Arecv,B),C,root=0)
end = MPI.Wtime()
print("Time taken by worker: ", rank,"is: ",end-start)
   

Time taken by worker:  0 is:  47.85671798399994
